In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.metrics import *

In [41]:
deliveries = pd.read_csv("deliveries.csv")
matches = pd.read_csv("matches.csv")

list_of_teams = np.unique(np.array(deliveries["batting_team"].unique().tolist() + deliveries["bowling_team"].unique().tolist()))

deliveries["batting_team"] = deliveries["batting_team"].replace("Deccan Chargers", "Sunrisers Hyderabad")
deliveries["bowling_team"] = deliveries["bowling_team"].replace("Deccan Chargers", "Sunrisers Hyderabad")

deliveries["batting_team"] = deliveries["batting_team"].replace("Pune Warriors", "Rising Pune Supergiants")
deliveries["bowling_team"] = deliveries["bowling_team"].replace("Pune Warriors", "Rising Pune Supergiants")

deliveries["batting_team"] = deliveries["batting_team"].replace("Rising Pune Supergiant", "Rising Pune Supergiants")
deliveries["bowling_team"] = deliveries["bowling_team"].replace("Rising Pune Supergiant", "Rising Pune Supergiants")

deliveries["batting_team"] = deliveries["batting_team"].replace("Delhi Daredevils", "Delhi Capitals")
deliveries["bowling_team"] = deliveries["bowling_team"].replace("Delhi Daredevils", "Delhi Capitals")

deliveries["batting_team"] = deliveries["batting_team"].replace("Kings XI Punjab", "Punjab Kings")
deliveries["bowling_team"] = deliveries["bowling_team"].replace("Kings XI Punjab", "Punjab Kings")

deliveries["batting_team"] = deliveries["batting_team"].replace("Gujarat Lions", "Gujarat Titans")
deliveries["bowling_team"] = deliveries["bowling_team"].replace("Gujarat Lions", "Gujarat Titans")

deliveries["batting_team"] = deliveries["batting_team"].str.title()
deliveries["bowling_team"] = deliveries["bowling_team"].str.title()

list_of_teams = np.unique(np.array(deliveries["batting_team"].unique().tolist() + deliveries["bowling_team"].unique().tolist()))

print(list_of_teams)
print(len(list_of_teams))

print()

['Chennai Super Kings' 'Delhi Capitals' 'Gujarat Titans'
 'Kochi Tuskers Kerala' 'Kolkata Knight Riders' 'Mumbai Indians'
 'Punjab Kings' 'Rajasthan Royals' 'Rising Pune Supergiants'
 'Royal Challengers Bangalore' 'Sunrisers Hyderabad']
11



In [42]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [59]:
df = deliveries.groupby(by=["match_id","batting_team","bowling_team"])["total_runs"].sum()
df = pd.DataFrame(df).reset_index().set_index("match_id")

scores = pd.DataFrame(columns=["match_id","Teams","Toss Winner","Toss Decision","Team 1","Team 2","Team 1 Score","Team 2 Score","Team 1 Wickets","Team 2 Wickets"])
team_scores = {}
ctr = 0

df.head()

indices = df.index.values
indices_unique = np.unique(indices)


for i in indices_unique:
    df_req = df.loc[i,:]
    count_of_indices = np.sum(df_req.index.values == i)
    if count_of_indices == 2:
        teams = df.loc[i,"batting_team"].values
        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        toss_decision = matches[matches["id"]==i]["toss_decision"].values[0]

        position_of_toss_winner = np.where(teams == toss_winner)[0][0]

        if toss_winner == teams[0]:
            if toss_decision == "bat":
                first_batting_team = teams[0]
                second_batting_team = teams[1]
            elif toss_decision == "field":
                first_batting_team = teams[1]
                second_batting_team = teams[0]
        else:
            if toss_decision == "bat":
                first_batting_team = teams[1]
                second_batting_team = teams[0]
            elif toss_decision == "field":
                first_batting_team = teams[0]
                second_batting_team = teams[1]
        
        p = teams[0] + " vs " + teams[1]

        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = p
        scores.loc[ctr,"Team 1"] = first_batting_team
        scores.loc[ctr,"Team 2"] = second_batting_team

        scores.loc[ctr,"Team 1 Score"] = df_req[df_req["batting_team"]==first_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Team 2 Score"] = df_req[df_req["batting_team"]==second_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = toss_decision

        ctr += 1
        print

IndexError: index 0 is out of bounds for axis 0 with size 0

In [58]:
scores

,match_id,Teams,Toss Winner,Toss Decision,Team 1,Team 2,Team 1 Score,Team 2 Score,Team 1 Wickets,Team 2 Wickets
0,1,Royal Challengers Bangalore vs Sunrisers Hyder...,Royal Challengers Bangalore,field,Sunrisers Hyderabad,Royal Challengers Bangalore,207,172,NaN,NaN


In [36]:
matches.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [8]:
df = deliveries.groupby(by=["match_id","batting_team","bowling_team"])["total_runs"].sum()
df = pd.DataFrame(df).reset_index().set_index("match_id")


scores = pd.DataFrame(columns=["match_id","Teams","Toss Winner","Toss Decision","Team 1","Team 2","Team 1 Score","Team 2 Score","Team 1 Wickets","Team 2 Wickets"])
team_scores = {}
ctr = 0
for i in df.index.unique():
    #print(df.loc[i,"batting_team"])
    try:
        teams = df.loc[i,"batting_team"].values
        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        toss_win = teams[position_of_toss_winner]

        if toss_winner == teams[0]:
            if choice == "bat":
                first_batting_team = teams[0]
            elif choice == "field":
                first_batting_team = teams[1]
        else:
            if choice == "bat":
                first_batting_team = teams[1]
            elif choice == "field":
                first_batting_team = teams[0]
        second_batting_team = teams[1-np.where(teams==first_batting_team)[0][0]]
        p = teams[0] + " vs " + teams[1]
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = p
        scores.loc[ctr,"Team 1"] = first_batting_team
        scores.loc[ctr,"Team 2"] = second_batting_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match[particular_match["batting_team"]==first_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Team 2 Score"] = particular_match[particular_match["batting_team"]==second_batting_team]["total_runs"].values[0]
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        #print(p)
        ctr += 1
        #break
    #print()
    except:
        print(ctr)
        batting_team = df.loc[i,"batting_team"]
        bowling_team = df.loc[i,"bowling_team"]
        print(batting_team)
        print(bowling_team)

        toss_winner = matches[matches["id"]==i]["toss_winner"].values[0]
        choice = matches[matches["id"]==i]["toss_decision"].values[0]
        #print("Toss Winning Team : ",toss_winner)
        #print("Choice of",toss_winner,":",choice)

        #position_of_toss_winner = np.where(teams == toss_winner)[0][0]
        #toss_win = teams[position_of_toss_winner]
        
        scores.loc[ctr,"match_id"] = i
        scores.loc[ctr,"Teams"] = batting_team + " vs " + bowling_team
        scores.loc[ctr,"Team 1"] = batting_team
        scores.loc[ctr,"Team 2"] = bowling_team
        particular_match = df.loc[i,]
        scores.loc[ctr,"Team 1 Score"] = particular_match["total_runs"]
        scores.loc[ctr,"Team 2 Score"] = np.nan
        scores.loc[ctr,"Toss Winner"] = toss_winner
        scores.loc[ctr,"Toss Decision"] = choice
        ctr += 1

1
match_id
2             Mumbai Indians
2    Rising Pune Supergiants
Name: batting_team, dtype: object
match_id
2    Rising Pune Supergiants
2             Mumbai Indians
Name: bowling_team, dtype: object


ValueError: Incompatible indexer with Series

In [100]:
wickets = pd.DataFrame(columns=["Team 1 Wickets","Team 2 Wickets"])
ctr = 0
for i in deliveries["match_id"].unique():
    wickets_first_team = deliveries[deliveries["match_id"]==i][deliveries["inning"]==1]["player_dismissed"].count()
    wickets.loc[ctr,"Team 1 Wickets"] = wickets_first_team
    try:
        wickets_second_team = deliveries[deliveries["match_id"]==i][deliveries["inning"]==2]["player_dismissed"].count()
        wickets.loc[ctr,"Team 2 Wickets"] = wickets_second_team
    except:
        print()
    ctr += 1

scores["Team 1 Wickets"] = wickets["Team 1 Wickets"].copy()
scores["Team 2 Wickets"] = wickets["Team 2 Wickets"].copy()
scores["city"] = matches["city"].copy()


In [101]:
deli = deliveries[["match_id","inning","over","ball"]].drop_duplicates()
deli = deli[deli["ball"]<=6].groupby(by=["match_id","inning"])["ball"].count()
deli = deli.reset_index()
dgh = pd.DataFrame(columns=["match_id","Team 1 Overs","Team 1 Balls","Team 2 Overs","Team 2 Balls"])
ctr = 0
for i in deli["match_id"].unique():
    dgh.loc[ctr,"match_id"] = i

    total_balls_first_team = deli[deli["match_id"]==i]["ball"].values[0]

    overs_first_team = int(np.floor(total_balls_first_team / 6))
    balls_first_team = total_balls_first_team - overs_first_team * 6

    dgh.loc[ctr,"Team 1 Overs"] = overs_first_team
    dgh.loc[ctr,"Team 1 Balls"] = balls_first_team
    

    try:
        total_balls_second_team = deli[deli["match_id"]==i]["ball"].values[1]

        overs_second_team = int(np.floor(total_balls_second_team / 6))
        balls_second_team = total_balls_second_team - overs_second_team * 6

        dgh.loc[ctr,"Team 2 Overs"] = overs_second_team
        dgh.loc[ctr,"Team 2 Balls"] = balls_second_team
    except:
        print()
    ctr += 1

dgh.head()

scores[["Team 1 Overs","Team 1 Balls","Team 2 Overs","Team 2 Balls"]] = dgh[["Team 1 Overs","Team 1 Balls","Team 2 Overs","Team 2 Balls"]].copy()

scores["Target"] = scores["Team 1 Score"] + 1
scores.head()

,match_id,Teams,Toss Winner,Toss Decision,Team 1,Team 2,Team 1 Score,Team 2 Score,Team 1 Wickets,Team 2 Wickets,city,Team 1 Overs,Team 1 Balls,Team 2 Overs,Team 2 Balls,Target
0,1,Royal Challengers Bangalore vs Sunrisers Hyder...,Royal Challengers Bangalore,field,Sunrisers Hyderabad,Royal Challengers Bangalore,207,172,4,10,Hyderabad,20,0,19,4,208
1,2,Mumbai Indians vs Rising Pune Supergiant,Rising Pune Supergiant,field,Mumbai Indians,Rising Pune Supergiant,184,187,8,3,Pune,20,0,19,5,185
2,3,Gujarat Lions vs Kolkata Knight Riders,Kolkata Knight Riders,field,Gujarat Lions,Kolkata Knight Riders,183,184,4,0,Rajkot,20,0,15,0,184
3,4,Kings XI Punjab vs Rising Pune Supergiant,Kings XI Punjab,field,Rising Pune Supergiant,Kings XI Punjab,163,164,6,4,Indore,20,0,19,0,164
4,5,Delhi Daredevils vs Royal Challengers Bangalore,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,Delhi Daredevils,157,142,8,9,Bangalore,20,0,20,0,158


In [102]:
for i in scores.index:
    if scores.loc[i,"Team 1 Score"] > scores.loc[i,"Team 2 Score"]:
        scores.loc[i,"Team 1 Win Probability"] = 100
    else:
        scores.loc[i,"Team 1 Win Probability"] = 0

In [103]:
# df_1.head()

In [104]:
df_1 = scores[["Team 1", "Team 2", "city", "Target", "Team 2 Score", "Team 2 Overs", "Team 2 Balls", "Team 2 Wickets","Team 1 Win Probability"]]



int_columns = ["Target", "Team 2 Score", "Team 2 Overs", "Team 2 Balls", "Team 2 Wickets","Team 1 Win Probability"]

for i in int_columns:
    df_1[i] = df_1[i].fillna(0)
    df_1[i] = df_1[i].astype("int64")

df_1["city"] = df_1["city"].fillna("Not specified")

In [105]:
ohe = OneHotEncoder(drop="first")
#print(df_1.dtypes)

cols = df_1.dtypes[df_1.dtypes=="object"].index.values.tolist()

df_2 = df_1.copy()


one_hot_encoded = ohe.fit_transform(df_2[cols])

df_3 = pd.DataFrame(ohe.fit_transform(df_2[cols]).toarray(),columns=ohe.get_feature_names_out(cols))

df_3[int_columns] = df_2[int_columns]

df_3.head()

,Team 1_Deccan Chargers,Team 1_Delhi Capitals,Team 1_Delhi Daredevils,Team 1_Gujarat Lions,Team 1_Kings XI Punjab,Team 1_Kochi Tuskers Kerala,Team 1_Kolkata Knight Riders,Team 1_Mumbai Indians,Team 1_Pune Warriors,Team 1_Rajasthan Royals,...,city_Rajkot,city_Ranchi,city_Sharjah,city_Visakhapatnam,Target,Team 2 Score,Team 2 Overs,Team 2 Balls,Team 2 Wickets,Team 1 Win Probability
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,208,172,19,4,10,100
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,185,187,19,5,3,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,184,184,15,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,164,164,19,0,4,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,158,142,20,0,9,100


In [110]:
X = df_3.drop("Team 1 Win Probability",axis=1)
y = df_3["Team 1 Win Probability"]

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=np.random.randint(0,1000),test_size=0.2)

rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [112]:
y_pred = rfc.predict(X_test)


scores_final_df_1 = pd.DataFrame(X_test,columns=X.columns)
scores_final_df_1["Actual Team 1 Win Probability"] = y_test
scores_final_df_1["Predicted Team 1 Win Probability"] = y_pred
scores_final_df_1["Actual Team 2 Win Probability"] = 100 - y_test
scores_final_df_1["Predicted Team 2 Win Probability"] = 100 - y_pred
scores_final_df_1.head()

,Team 1_Deccan Chargers,Team 1_Delhi Capitals,Team 1_Delhi Daredevils,Team 1_Gujarat Lions,Team 1_Kings XI Punjab,Team 1_Kochi Tuskers Kerala,Team 1_Kolkata Knight Riders,Team 1_Mumbai Indians,Team 1_Pune Warriors,Team 1_Rajasthan Royals,...,city_Visakhapatnam,Target,Team 2 Score,Team 2 Overs,Team 2 Balls,Team 2 Wickets,Actual Team 1 Win Probability,Predicted Team 1 Win Probability,Actual Team 2 Win Probability,Predicted Team 2 Win Probability
472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,126,126,18,5,4,0,0,100,100
323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,156,156,19,2,3,0,0,100,100
156,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,120,122,16,2,2,0,0,100,100
717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,159,152,19,5,4,100,0,0,100
707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,183,174,20,0,8,100,100,0,0


In [124]:
accu = accuracy_score(scores_final_df_1["Actual Team 1 Win Probability"],scores_final_df_1["Predicted Team 1 Win Probability"])
print(np.round(accu * 100,2))

90.13


In [125]:
X = df_3.drop("Team 1 Win Probability",axis=1)
y = df_3["Team 1 Win Probability"]

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=np.random.randint(0,1000),test_size=0.2)

rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)

RandomForestRegressor()

In [127]:
y_pred = rfr.predict(X_test)


scores_final_df = pd.DataFrame(X_test,columns=X.columns)
scores_final_df["Actual Team 1 Win Probability"] = y_test
scores_final_df["Predicted Team 1 Win Probability"] = y_pred
scores_final_df["Actual Team 2 Win Probability"] = 100 - y_test
scores_final_df["Predicted Team 2 Win Probability"] = 100 - y_pred
scores_final_df.head()

,Team 1_Deccan Chargers,Team 1_Delhi Capitals,Team 1_Delhi Daredevils,Team 1_Gujarat Lions,Team 1_Kings XI Punjab,Team 1_Kochi Tuskers Kerala,Team 1_Kolkata Knight Riders,Team 1_Mumbai Indians,Team 1_Pune Warriors,Team 1_Rajasthan Royals,...,city_Visakhapatnam,Target,Team 2 Score,Team 2 Overs,Team 2 Balls,Team 2 Wickets,Actual Team 1 Win Probability,Predicted Team 1 Win Probability,Actual Team 2 Win Probability,Predicted Team 2 Win Probability
714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,143,102,17,4,10,100,74.0,0,26.0
290,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,90,105,12,4,6,0,29.0,100,71.0
605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,186,189,19,1,5,0,1.0,100,99.0
203,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,182,157,20,0,5,100,96.0,0,4.0
153,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,124,125,17,1,3,0,5.0,100,95.0


In [ ]:
po = 

In [131]:
X_test.iloc[0,].index

Index(['Team 1_Deccan Chargers', 'Team 1_Delhi Capitals',
       'Team 1_Delhi Daredevils', 'Team 1_Gujarat Lions',
       'Team 1_Kings XI Punjab', 'Team 1_Kochi Tuskers Kerala',
       'Team 1_Kolkata Knight Riders', 'Team 1_Mumbai Indians',
       'Team 1_Pune Warriors', 'Team 1_Rajasthan Royals',
       'Team 1_Rising Pune Supergiant', 'Team 1_Rising Pune Supergiants',
       'Team 1_Royal Challengers Bangalore', 'Team 1_Sunrisers Hyderabad',
       'Team 2_Deccan Chargers', 'Team 2_Delhi Capitals',
       'Team 2_Delhi Daredevils', 'Team 2_Gujarat Lions',
       'Team 2_Kings XI Punjab', 'Team 2_Kochi Tuskers Kerala',
       'Team 2_Kolkata Knight Riders', 'Team 2_Mumbai Indians',
       'Team 2_Pune Warriors', 'Team 2_Rajasthan Royals',
       'Team 2_Rising Pune Supergiant', 'Team 2_Rising Pune Supergiants',
       'Team 2_Royal Challengers Bangalore', 'Team 2_Sunrisers Hyderabad',
       'city_Ahmedabad', 'city_Bangalore', 'city_Bengaluru',
       'city_Bloemfontein', 'city_Cap

In [129]:
X_test.iloc[0,]

Team 1_Deccan Chargers       0.0
Team 1_Delhi Capitals        0.0
Team 1_Delhi Daredevils      0.0
Team 1_Gujarat Lions         0.0
Team 1_Kings XI Punjab       0.0
                           ...  
Target                     143.0
Team 2 Score               102.0
Team 2 Overs                17.0
Team 2 Balls                 4.0
Team 2 Wickets              10.0
Name: 714, Length: 65, dtype: float64